# Problem 3

In [ ]:
# Runs qml.clifford_t_decomposition to approximate 
# exponential of a Pauli string: exp(i(π/7)Z ⊗ Z) = (CNOT1,2)*(I x Rz(-2pi/7)*(CNOT1,2)
# to error threshold (epsilon = 1e-6)

import pennylane as qml
import numpy as np

theta = -2 * np.pi / 7
wire = 1  # Emit decomposition on q[1]

def expand_to_HT(op):
    w = op.wires[0]
    name = op.name

    # Allowed basis directly
    if name == "Hadamard":
        return [qml.Hadamard(wires=w)]
    if name == "T":
        return [qml.T(wires=w)]
    if name in ("Adjoint(T)", "T.inv", "Tdg"):
        return [qml.adjoint(qml.T)(wires=w)]

    # S = T^2
    if name == "S":
        return [qml.T(wires=w), qml.T(wires=w)]

    # Z = T^4
    if name in ("PauliZ", "Z"):
        return [qml.T(wires=w)] * 4

    # X = H Z H = H T^4 H
    if name in ("PauliX", "X"):
        return [qml.Hadamard(wires=w), *([qml.T(wires=w)] * 4), qml.Hadamard(wires=w)]

    # Ignore global phase: Y ~ X Z = (H T^4 H)(T^4)
    if name in ("PauliY", "Y"):
        return [
            qml.Hadamard(wires=w),
            *([qml.T(wires=w)] * 4),
            qml.Hadamard(wires=w),
            *([qml.T(wires=w)] * 4),
        ]

    # SX ~ H S H = H (T^2) H
    if name == "SX":
        return [qml.Hadamard(wires=w), qml.T(wires=w), qml.T(wires=w), qml.Hadamard(wires=w)]

    # S† = T† T†
    if name in ("Adjoint(S)", "S.inv", "Sdg"):
        return [qml.adjoint(qml.T)(wires=w), qml.adjoint(qml.T)(wires=w)]

    raise ValueError(f"Don't know how to rewrite op '{name}' into {{H, T, Tdg}}")

# Build tape with RZ(theta)
tape = qml.tape.QuantumScript(
    ops=[qml.RZ(theta, wires=wire)],
    measurements=[]
)

# Approx Clifford+T Decomp
(ct_batch, _postproc) = qml.clifford_t_decomposition(
    tape,
    method="gridsynth",
    epsilon=1e-6,
)
ct_tape = ct_batch[0]

# Rewrite to HT-only
ht_ops = []
for op in ct_tape.operations:
    ht_ops.extend(expand_to_HT(op))

# Output OpenQASM 2.0
def op_to_qasm_line(op):
    """Map PennyLane op -> OpenQASM line."""
    w = int(op.wires[0])
    if op.name == "Hadamard":
        return f"h q[{w}];"
    if op.name == "T":
        return f"t q[{w}];"
    if op.name in ("Adjoint(T)", "T.inv", "Tdg"):
        return f"tdg q[{w}];"
    raise ValueError(f"Unexpected op in HT-only list: {op.name}")

qasm_lines = []
qasm_lines.append("OPENQASM 2.0;")
qasm_lines.append('include "qelib1.inc";')
qasm_lines.append("")
qasm_lines.append("qreg q[2];")
qasm_lines.append("")
qasm_lines.append("cx q[0],q[1];")

# Decomposition on q[1]
for op in ht_ops:
    qasm_lines.append(op_to_qasm_line(op))

qasm_lines.append("cx q[0],q[1];")
qasm_lines.append("")

out_path = "rz_minus_2pi_over_7_with_cx.qasm"
with open(out_path, "w", encoding="utf-8") as f:
    f.write("\n".join(qasm_lines))

print(f"Wrote QASM to: {out_path}")
print("Gate count in HT-only RZ decomposition:", len(ht_ops))

Wrote QASM to: rz_minus_2pi_over_7_with_cx.qasm
Gate count in HT-only RZ decomposition: 234


# Check Result

In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

# Load QASM circuit
qc = QuantumCircuit.from_qasm_file("rz_minus_2pi_over_7_with_cx.qasm")
U_qasm = Operator(qc).data

# Build target unitary
Z = np.array([[1, 0],
              [0, -1]], dtype=complex)

phi = np.pi / 7
ZZ = np.kron(Z, Z)

I4 = np.eye(4, dtype=complex)
U_target = np.cos(phi) * I4 + 1j * np.sin(phi) * ZZ

# Compare up to global phase
phase = np.vdot(U_qasm.flatten(), U_target.flatten())  # <U_qasm, U_target>
U_qasm_phase_fixed = U_qasm * np.exp(-1j * np.angle(phase))

print("Allclose up to global phase:", np.allclose(U_qasm_phase_fixed, U_target, atol=1e-8))

# Maximum entrywise error
err = np.max(np.abs(U_qasm_phase_fixed - U_target))
print("Max |Δ| after global-phase fix:", err)


Allclose up to global phase: False
Max |Δ| after global-phase fix: 2.660366300887575e-07
